In [1]:
#Import Libraries

import pandas as  pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
from datetime import datetime 
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, accuracy_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings

In [2]:
#%%capture 
#!pip install catboost

In [3]:
train = pd.read_csv('train.csv')
test =  pd.read_csv('test.csv')

In [4]:
#create submision file
submit = pd.DataFrame(columns = ['ID', 'Response'])
submit.ID = test.ID
submit.head()

,ID,Response
0,ID_ZPMABNVX,NaN
1,ID_WFE91NAA,NaN
2,ID_JV11RBRK,NaN
3,ID_6B7SVKY9,NaN
4,ID_GOVUZ545,NaN


In [5]:
train.head()

,ID,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,...,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,Response
0,ID_4A9AR6FQ,1955,Graduation,Divorced,77504.4,1,1,22-06-2014,56,2,...,6,20,3,0,0,0,0,0,0,0
1,ID_X28T3VEK,1958,Graduation,Together,56784.0,0,1,01-08-2013,17,6,...,15,19,6,0,0,0,0,0,0,0
2,ID_AWXARH57,1962,Graduation,Single,103714.8,0,0,21-11-2013,17,0,...,18,37,2,0,0,0,1,0,0,1
3,ID_FQVZHE81,1979,2n Cycle,Single,46311.6,1,0,13-09-2013,49,3,...,24,24,8,0,0,0,0,0,0,0
4,ID_QVLWGPRN,1959,Graduation,Married,87486.0,0,0,21-01-2014,59,1,...,165,203,1,0,0,0,1,0,0,0


In [6]:
test.head()

,ID,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,...,Amount_on_FishProducts,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain
0,ID_ZPMABNVX,1954,Graduation,Single,48556.8,0,1,11-01-2013,85,6,...,7,23,41,8,0,0,0,0,0,0
1,ID_WFE91NAA,1961,Graduation,Widow,57499.2,0,1,22-11-2012,79,5,...,0,0,75,6,0,1,0,0,0,0
2,ID_JV11RBRK,1973,Basic,Married,17025.6,0,0,28-02-2013,47,1,...,16,12,27,6,0,0,0,0,0,0
3,ID_6B7SVKY9,1970,Graduation,Together,91983.6,0,0,16-08-2013,98,1,...,219,189,126,2,0,0,1,1,0,0
4,ID_GOVUZ545,1959,Graduation,Together,78235.2,0,2,25-07-2013,41,2,...,12,0,200,5,1,0,0,0,0,0


In [7]:
train.shape

(1568, 27)

In [8]:
train.isnull().sum()

ID                          0
Year_of_Birth               0
Education_Level             0
Marital_Status              0
Disposable_Income          16
No_of_Kids_in_home          0
No_of_Teen_in_home          0
Date_Customer               0
Recency                     0
Discounted_Purchases        0
WebPurchases                0
CatalogPurchases            0
StorePurchases              0
Amount_on_Wines             0
Amount_on_Fruits            0
Amount_on_MeatProducts      0
Amount_on_FishProducts      0
Amount_on_SweetProducts     0
Amount_on_GoldProds         0
WebVisitsMonth              0
Cmp3Accepted                0
Cmp4Accepted                0
Cmp5Accepted                0
Cmp1Accepted                0
Cmp2Accepted                0
Any_Complain                0
Response                    0
dtype: int64

In [9]:
train.dtypes

ID                          object
Year_of_Birth                int64
Education_Level             object
Marital_Status              object
Disposable_Income          float64
No_of_Kids_in_home           int64
No_of_Teen_in_home           int64
Date_Customer               object
Recency                      int64
Discounted_Purchases         int64
WebPurchases                 int64
CatalogPurchases             int64
StorePurchases               int64
Amount_on_Wines              int64
Amount_on_Fruits             int64
Amount_on_MeatProducts       int64
Amount_on_FishProducts       int64
Amount_on_SweetProducts      int64
Amount_on_GoldProds          int64
WebVisitsMonth               int64
Cmp3Accepted                 int64
Cmp4Accepted                 int64
Cmp5Accepted                 int64
Cmp1Accepted                 int64
Cmp2Accepted                 int64
Any_Complain                 int64
Response                     int64
dtype: object

In [10]:
test.dtypes

ID                          object
Year_of_Birth                int64
Education_Level             object
Marital_Status              object
Disposable_Income          float64
No_of_Kids_in_home           int64
No_of_Teen_in_home           int64
Date_Customer               object
Recency                      int64
Discounted_Purchases         int64
WebPurchases                 int64
CatalogPurchases             int64
StorePurchases               int64
Amount_on_Wines              int64
Amount_on_Fruits             int64
Amount_on_MeatProducts       int64
Amount_on_FishProducts       int64
Amount_on_SweetProducts      int64
Amount_on_GoldProds          int64
WebVisitsMonth               int64
Cmp3Accepted                 int64
Cmp4Accepted                 int64
Cmp5Accepted                 int64
Cmp1Accepted                 int64
Cmp2Accepted                 int64
Any_Complain                 int64
dtype: object

In [11]:
train.drop('ID', axis = 1,inplace=True)
test.drop('ID', axis = 1,inplace=True)

In [12]:
train.Date_Customer = pd.to_datetime(train['Date_Customer'])

In [13]:
test.Date_Customer = pd.to_datetime(test['Date_Customer'])

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year_of_Birth            1568 non-null   int64         
 1   Education_Level          1568 non-null   object        
 2   Marital_Status           1568 non-null   object        
 3   Disposable_Income        1552 non-null   float64       
 4   No_of_Kids_in_home       1568 non-null   int64         
 5   No_of_Teen_in_home       1568 non-null   int64         
 6   Date_Customer            1568 non-null   datetime64[ns]
 7   Recency                  1568 non-null   int64         
 8   Discounted_Purchases     1568 non-null   int64         
 9   WebPurchases             1568 non-null   int64         
 10  CatalogPurchases         1568 non-null   int64         
 11  StorePurchases           1568 non-null   int64         
 12  Amount_on_Wines          1568 non-

In [15]:
#let's  sepparate the columns by type
cat_cols = train.select_dtypes(include=['object','category']).columns

num_cols = [col for col in train.columns if col not in cat_cols]

num_cols

['Year_of_Birth',
 'Disposable_Income',
 'No_of_Kids_in_home',
 'No_of_Teen_in_home',
 'Date_Customer',
 'Recency',
 'Discounted_Purchases',
 'WebPurchases',
 'CatalogPurchases',
 'StorePurchases',
 'Amount_on_Wines',
 'Amount_on_Fruits',
 'Amount_on_MeatProducts',
 'Amount_on_FishProducts',
 'Amount_on_SweetProducts',
 'Amount_on_GoldProds',
 'WebVisitsMonth',
 'Cmp3Accepted',
 'Cmp4Accepted',
 'Cmp5Accepted',
 'Cmp1Accepted',
 'Cmp2Accepted',
 'Any_Complain',
 'Response']

In [16]:
cat_cols

Index(['Education_Level', 'Marital_Status'], dtype='object')

In [17]:
for col in cat_cols: 
    print (f'{col} :  {train[col].nunique()}')
    print(f'and the unique entries are: {train[col].unique()}')

Education_Level :  5
and the unique entries are: ['Graduation' '2n Cycle' 'Master' 'PhD' 'Basic']
Marital_Status :  8
and the unique entries are: ['Divorced' 'Together' 'Single' 'Married' 'Widow' 'Alone' 'Absurd' 'YOLO']


In [18]:
for col in cat_cols: 
    print (f'{col} :  {test[col].nunique()}')
    print(f'and the unique entries are: {test[col].unique()}')

Education_Level :  5
and the unique entries are: ['Graduation' 'Basic' 'Master' '2n Cycle' 'PhD']
Marital_Status :  6
and the unique entries are: ['Single' 'Widow' 'Married' 'Together' 'Divorced' 'Absurd']


In [19]:
#Fill missing values. Let's fill with mean 
train.Disposable_Income.fillna((train.Disposable_Income.mean()), inplace = True)
train.isnull().sum()

Year_of_Birth              0
Education_Level            0
Marital_Status             0
Disposable_Income          0
No_of_Kids_in_home         0
No_of_Teen_in_home         0
Date_Customer              0
Recency                    0
Discounted_Purchases       0
WebPurchases               0
CatalogPurchases           0
StorePurchases             0
Amount_on_Wines            0
Amount_on_Fruits           0
Amount_on_MeatProducts     0
Amount_on_FishProducts     0
Amount_on_SweetProducts    0
Amount_on_GoldProds        0
WebVisitsMonth             0
Cmp3Accepted               0
Cmp4Accepted               0
Cmp5Accepted               0
Cmp1Accepted               0
Cmp2Accepted               0
Any_Complain               0
Response                   0
dtype: int64

In [20]:
test.Disposable_Income.fillna((test.Disposable_Income.mean()), inplace = True)
test.isnull().sum()

Year_of_Birth              0
Education_Level            0
Marital_Status             0
Disposable_Income          0
No_of_Kids_in_home         0
No_of_Teen_in_home         0
Date_Customer              0
Recency                    0
Discounted_Purchases       0
WebPurchases               0
CatalogPurchases           0
StorePurchases             0
Amount_on_Wines            0
Amount_on_Fruits           0
Amount_on_MeatProducts     0
Amount_on_FishProducts     0
Amount_on_SweetProducts    0
Amount_on_GoldProds        0
WebVisitsMonth             0
Cmp3Accepted               0
Cmp4Accepted               0
Cmp5Accepted               0
Cmp1Accepted               0
Cmp2Accepted               0
Any_Complain               0
dtype: int64

In [21]:
print(f'The Size of the train set is now: {train.shape}')
#train.head()

The Size of the train set is now: (1568, 26)


In [22]:
print(f'The Size of the test set is now: {test.shape}')

The Size of the test set is now: (672, 25)


In [23]:
#let\s encode the categorical variable. 
#We'll use label encoding because the unique values in the test andtrain dataset aren't the same

label_encode = LabelEncoder()
for i in cat_cols:
    train[i] =  label_encode.fit_transform(train[i])
    test[i] = label_encode.fit_transform(test[i])
    
train.head()

,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,WebPurchases,...,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,Response
0,1955,2,2,77504.4,1,1,2014-06-22,56,2,1,...,6,20,3,0,0,0,0,0,0,0
1,1958,2,5,56784.0,0,1,2013-01-08,17,6,5,...,15,19,6,0,0,0,0,0,0,0
2,1962,2,4,103714.8,0,0,2013-11-21,17,0,7,...,18,37,2,0,0,0,1,0,0,1
3,1979,0,4,46311.6,1,0,2013-09-13,49,3,4,...,24,24,8,0,0,0,0,0,0,0
4,1959,2,3,87486.0,0,0,2014-01-21,59,1,3,...,165,203,1,0,0,0,1,0,0,0


In [34]:
#let's try some modelling for initial submission

# split tox and y 
y = train.Response
X = train.drop('Response', axis = 1)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size= 0.3, random_state= 3)

In [36]:
cross_val = KFold(n_splits=10, random_state=1, shuffle=True)

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

NameError: name 'model' is not defined

In [39]:
scores = cross_val_score(cat_model, X, y, scoring='f1_macro', cv=cross_val, n_jobs=-1)

NameError: name 'cat_model' is not defined

In [ ]:
scores

In [ ]:
#!pip install lightgbm

In [ ]:
#from xgboost import XGBRegressor
#xgb = XGBRegressor(random_state=5, n_jobs = -1)
#xgb.fit(X, y)
#y_pred_xgb = xgb.predict(X_test)
#print("xgb F1 score on validation set is : ",f1_score(y_test,y_pred_xgb))

In [40]:
import lightgbm as lgm
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier()
cat_model.fit(X, y)
y_pred_cat = cat_model.predict(X_test)
print("Catboost F1 score on validation set is : ",f1_score(y_test,y_pred_cat))

Learning rate set to 0.012484
0:	learn: 0.6817859	total: 179ms	remaining: 2m 58s
1:	learn: 0.6720226	total: 188ms	remaining: 1m 33s
2:	learn: 0.6606450	total: 196ms	remaining: 1m 5s
3:	learn: 0.6512627	total: 203ms	remaining: 50.6s
4:	learn: 0.6413627	total: 208ms	remaining: 41.5s
5:	learn: 0.6311174	total: 215ms	remaining: 35.6s
6:	learn: 0.6233591	total: 220ms	remaining: 31.2s
7:	learn: 0.6140929	total: 226ms	remaining: 28s
8:	learn: 0.6066167	total: 232ms	remaining: 25.6s
9:	learn: 0.5991645	total: 237ms	remaining: 23.5s
10:	learn: 0.5929789	total: 243ms	remaining: 21.8s
11:	learn: 0.5858551	total: 249ms	remaining: 20.5s
12:	learn: 0.5784402	total: 254ms	remaining: 19.3s
13:	learn: 0.5702485	total: 261ms	remaining: 18.4s
14:	learn: 0.5629452	total: 296ms	remaining: 19.5s
15:	learn: 0.5567017	total: 301ms	remaining: 18.5s
16:	learn: 0.5491103	total: 305ms	remaining: 17.6s
17:	learn: 0.5425426	total: 311ms	remaining: 16.9s
18:	learn: 0.5359274	total: 315ms	remaining: 16.3s
19:	learn: 

180:	learn: 0.2427829	total: 881ms	remaining: 3.98s
181:	learn: 0.2421194	total: 886ms	remaining: 3.98s
182:	learn: 0.2415948	total: 889ms	remaining: 3.97s
183:	learn: 0.2412027	total: 894ms	remaining: 3.96s
184:	learn: 0.2408536	total: 898ms	remaining: 3.96s
185:	learn: 0.2405551	total: 901ms	remaining: 3.94s
186:	learn: 0.2402169	total: 905ms	remaining: 3.93s
187:	learn: 0.2397359	total: 907ms	remaining: 3.92s
188:	learn: 0.2393357	total: 911ms	remaining: 3.91s
189:	learn: 0.2388985	total: 914ms	remaining: 3.9s
190:	learn: 0.2382519	total: 918ms	remaining: 3.89s
191:	learn: 0.2376174	total: 921ms	remaining: 3.87s
192:	learn: 0.2371840	total: 924ms	remaining: 3.86s
193:	learn: 0.2365763	total: 928ms	remaining: 3.85s
194:	learn: 0.2361753	total: 931ms	remaining: 3.84s
195:	learn: 0.2354208	total: 935ms	remaining: 3.83s
196:	learn: 0.2349035	total: 938ms	remaining: 3.82s
197:	learn: 0.2345275	total: 942ms	remaining: 3.81s
198:	learn: 0.2341944	total: 945ms	remaining: 3.8s
199:	learn: 0.

387:	learn: 0.1761146	total: 1.57s	remaining: 2.48s
388:	learn: 0.1758498	total: 1.58s	remaining: 2.48s
389:	learn: 0.1756541	total: 1.58s	remaining: 2.47s
390:	learn: 0.1754264	total: 1.58s	remaining: 2.47s
391:	learn: 0.1752297	total: 1.59s	remaining: 2.46s
392:	learn: 0.1749598	total: 1.59s	remaining: 2.46s
393:	learn: 0.1746949	total: 1.59s	remaining: 2.45s
394:	learn: 0.1745334	total: 1.6s	remaining: 2.45s
395:	learn: 0.1743380	total: 1.6s	remaining: 2.44s
396:	learn: 0.1741732	total: 1.6s	remaining: 2.44s
397:	learn: 0.1738403	total: 1.61s	remaining: 2.43s
398:	learn: 0.1735528	total: 1.61s	remaining: 2.43s
399:	learn: 0.1732685	total: 1.61s	remaining: 2.42s
400:	learn: 0.1730919	total: 1.62s	remaining: 2.42s
401:	learn: 0.1729072	total: 1.62s	remaining: 2.41s
402:	learn: 0.1726347	total: 1.63s	remaining: 2.41s
403:	learn: 0.1724101	total: 1.63s	remaining: 2.4s
404:	learn: 0.1721427	total: 1.63s	remaining: 2.4s
405:	learn: 0.1718819	total: 1.64s	remaining: 2.39s
406:	learn: 0.171

546:	learn: 0.1453396	total: 2.1s	remaining: 1.74s
547:	learn: 0.1452057	total: 2.1s	remaining: 1.73s
548:	learn: 0.1450861	total: 2.1s	remaining: 1.73s
549:	learn: 0.1449006	total: 2.11s	remaining: 1.72s
550:	learn: 0.1447714	total: 2.11s	remaining: 1.72s
551:	learn: 0.1446448	total: 2.12s	remaining: 1.72s
552:	learn: 0.1444971	total: 2.12s	remaining: 1.71s
553:	learn: 0.1442948	total: 2.12s	remaining: 1.71s
554:	learn: 0.1441451	total: 2.12s	remaining: 1.7s
555:	learn: 0.1439758	total: 2.13s	remaining: 1.7s
556:	learn: 0.1437536	total: 2.13s	remaining: 1.7s
557:	learn: 0.1436281	total: 2.13s	remaining: 1.69s
558:	learn: 0.1434684	total: 2.14s	remaining: 1.69s
559:	learn: 0.1433056	total: 2.14s	remaining: 1.68s
560:	learn: 0.1432332	total: 2.14s	remaining: 1.68s
561:	learn: 0.1429993	total: 2.15s	remaining: 1.67s
562:	learn: 0.1428334	total: 2.15s	remaining: 1.67s
563:	learn: 0.1427051	total: 2.15s	remaining: 1.66s
564:	learn: 0.1425585	total: 2.16s	remaining: 1.66s
565:	learn: 0.1424

720:	learn: 0.1179953	total: 2.61s	remaining: 1.01s
721:	learn: 0.1178244	total: 2.62s	remaining: 1.01s
722:	learn: 0.1176410	total: 2.62s	remaining: 1s
723:	learn: 0.1174640	total: 2.62s	remaining: 1000ms
724:	learn: 0.1173343	total: 2.63s	remaining: 996ms
725:	learn: 0.1171561	total: 2.63s	remaining: 992ms
726:	learn: 0.1169973	total: 2.63s	remaining: 988ms
727:	learn: 0.1168447	total: 2.63s	remaining: 985ms
728:	learn: 0.1167895	total: 2.64s	remaining: 981ms
729:	learn: 0.1166433	total: 2.64s	remaining: 977ms
730:	learn: 0.1165197	total: 2.64s	remaining: 973ms
731:	learn: 0.1163160	total: 2.65s	remaining: 969ms
732:	learn: 0.1162040	total: 2.65s	remaining: 965ms
733:	learn: 0.1160657	total: 2.65s	remaining: 961ms
734:	learn: 0.1159845	total: 2.65s	remaining: 957ms
735:	learn: 0.1158850	total: 2.66s	remaining: 953ms
736:	learn: 0.1157235	total: 2.66s	remaining: 949ms
737:	learn: 0.1155867	total: 2.66s	remaining: 946ms
738:	learn: 0.1154072	total: 2.67s	remaining: 942ms
739:	learn: 0.

907:	learn: 0.0953849	total: 3.14s	remaining: 318ms
908:	learn: 0.0952285	total: 3.14s	remaining: 315ms
909:	learn: 0.0951224	total: 3.15s	remaining: 311ms
910:	learn: 0.0950684	total: 3.15s	remaining: 308ms
911:	learn: 0.0949574	total: 3.15s	remaining: 304ms
912:	learn: 0.0948405	total: 3.15s	remaining: 301ms
913:	learn: 0.0947551	total: 3.16s	remaining: 297ms
914:	learn: 0.0945917	total: 3.16s	remaining: 294ms
915:	learn: 0.0944831	total: 3.17s	remaining: 290ms
916:	learn: 0.0943603	total: 3.17s	remaining: 287ms
917:	learn: 0.0942324	total: 3.17s	remaining: 283ms
918:	learn: 0.0941094	total: 3.17s	remaining: 280ms
919:	learn: 0.0940402	total: 3.18s	remaining: 276ms
920:	learn: 0.0939116	total: 3.18s	remaining: 273ms
921:	learn: 0.0938271	total: 3.19s	remaining: 269ms
922:	learn: 0.0937247	total: 3.19s	remaining: 266ms
923:	learn: 0.0936850	total: 3.19s	remaining: 263ms
924:	learn: 0.0935958	total: 3.19s	remaining: 259ms
925:	learn: 0.0934532	total: 3.2s	remaining: 256ms
926:	learn: 0

In [41]:
cat_preds = cat_model.predict(test)

In [42]:
submit.Response = cat_preds
submit.head()

,ID,Response
0,ID_ZPMABNVX,0
1,ID_WFE91NAA,0
2,ID_JV11RBRK,0
3,ID_6B7SVKY9,1
4,ID_GOVUZ545,0


In [43]:
submit.to_csv('Second_Entry', index=False)